### load the data

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [94]:
train = pd.read_csv('covid.csv', low_memory=False)

In [142]:
train.tail()

,Id,County,Province_State,Country_Region,Population,Weight,Date,Target,TargetValue
969635,969636,NaN,NaN,Zimbabwe,14240168,0.607106,2020-06-08,Fatalities,0
969636,969637,NaN,NaN,Zimbabwe,14240168,0.060711,2020-06-09,ConfirmedCases,27
969637,969638,NaN,NaN,Zimbabwe,14240168,0.607106,2020-06-09,Fatalities,0
969638,969639,NaN,NaN,Zimbabwe,14240168,0.060711,2020-06-10,ConfirmedCases,6
969639,969640,NaN,NaN,Zimbabwe,14240168,0.607106,2020-06-10,Fatalities,0


### analyse the world trend

In [110]:
train_df = train.groupby(['Date', 'Target']).sum()['TargetValue'].reset_index()

In [138]:
train_df

,Date,Target,TargetValue
0,2020-01-23,ConfirmedCases,194
1,2020-01-23,Fatalities,2
2,2020-01-24,ConfirmedCases,564
3,2020-01-24,Fatalities,16
4,2020-01-25,ConfirmedCases,979
...,...,...,...
275,2020-06-08,Fatalities,4740
276,2020-06-09,ConfirmedCases,159427
277,2020-06-09,Fatalities,6900
278,2020-06-10,ConfirmedCases,160388


In [111]:
fig = px.line(train_df, x="Date", y="TargetValue", color='Target', 
              title="Worldwide Confirmed/Death Cases Over Time")
fig.show()

In [199]:
world_trend = train_df.set_index(['Date','Target']).unstack('Target').reset_index()
world_trend.columns = ['date', 'new_cases', 'new_deaths']

In [200]:
world_trend

,date,new_cases,new_deaths
0,2020-01-23,194,2
1,2020-01-24,564,16
2,2020-01-25,979,32
3,2020-01-26,1358,28
4,2020-01-27,1612,52
...,...,...,...
135,2020-06-06,173111,5341
136,2020-06-07,149845,3660
137,2020-06-08,140218,4740
138,2020-06-09,159427,6900


In [212]:
world_trend['growth_rate'] = world_trend['new_cases'] / world_trend['new_cases'].shift(1)
world_trend['total_cases'] = world_trend['new_cases'].cumsum()
world_trend['total_deaths'] = world_trend['new_deaths'].cumsum()
world_trend['mortality_rate'] = world_trend['total_deaths']/world_trend['total_cases'] 

In [213]:
world_trend

,date,new_cases,new_deaths,total_cases,growth_rate,total_deaths,mortality_rate
0,2020-01-23,194,2,194,NaN,2,0.010309
1,2020-01-24,564,16,758,2.907216,18,0.023747
2,2020-01-25,979,32,1737,1.735816,50,0.028785
3,2020-01-26,1358,28,3095,1.387130,78,0.025202
4,2020-01-27,1612,52,4707,1.187040,130,0.027618
...,...,...,...,...,...,...,...
135,2020-06-06,173111,5341,10918941,0.874590,631303,0.057817
136,2020-06-07,149845,3660,11068786,0.865601,634963,0.057365
137,2020-06-08,140218,4740,11209004,0.935754,639703,0.057070
138,2020-06-09,159427,6900,11368431,1.136994,646603,0.056877


In [214]:
world_trend_melt = pd.melt(world_trend, id_vars=['date'], value_vars=['total_cases', 'total_deaths', 'new_cases'])
world_trend_melt.head()

,date,variable,value
0,2020-01-23,total_cases,194
1,2020-01-24,total_cases,758
2,2020-01-25,total_cases,1737
3,2020-01-26,total_cases,3095
4,2020-01-27,total_cases,4707


In [215]:
fig = px.line(world_trend_melt, x='date', y='value', color='variable', title='covid19 worldwide trend over time')
fig.show()

In [220]:
fig = px.line(world_trend_melt, x='date', y='value', color='variable', title='covid19 worldwide trend over time (log_transfered)', log_y=True)
fig.show()

In [217]:
fig = px.line(world_trend, x='date', y='mortality_rate', title='world mortality rate over time')
fig.show()

In [219]:
fig = px.line(world_trend, x='date', y='growth_rate', title='world covid growth rate over time')
fig.update_yaxes(range=[0., 5.])
fig.show()